In [2]:
q='export PATH=/homes/lgaiser/miniconda3/envs/q2dev-amplicon/bin:$PATH'

In [3]:
! $q;  qiime dev refresh-cache

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


# Preprocessing

<font size="2"> <p> Importing reference hits from the first sequencing run </p> </font>

In [ ]:
! $q; qiime tools import \
  --type 'FeatureTable[Frequency]' \
  --input-path EDENISS/From_DADA2/first_run/feature_table_first_run_transposed.biom \
  --output-path EDENISS/Analysis_Paper_DADA2/feature_table_first_run.qza

In [ ]:
! $q; qiime tools import \
    --input-path EDENISS/From_DADA2/first_run/rep_seqs_first_run.fasta \
    --output-path EDENISS/Analysis_Paper_DADA2/rep_seqs_first_run.qza \
    --type 'FeatureData[Sequence]'

<font size="2"> <p> Importing reference hits from the missing samples in a second sequencing run </p> </font>

In [ ]:
! $q; qiime tools import \
  --type 'FeatureTable[Frequency]' \
  --input-path EDENISS/From_DADA2/second_run/feature_table_second_run_transposed.biom \
  --output-path EDENISS/Analysis_Paper_DADA2/feature_table_second_run.qza

In [ ]:
! $q; qiime tools import \
    --input-path EDENISS/From_DADA2/second_run/rep_seqs_second_run.fasta \
    --output-path EDENISS/Analysis_Paper_DADA2/rep_seqs_second_run.qza \
    --type 'FeatureData[Sequence]'

Import taxonomic annotation from DADA2 

In [ ]:
! $q; qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-path EDENISS/From_DADA2/first_run/taxonomy_first_run.tsv \
  --output-path EDENISS/Analysis_Paper_DADA2/taxonomy_first_run.qza \
  --input-format HeaderlessTSVTaxonomyFormat

In [ ]:
! $q; qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-path EDENISS/From_DADA2/second_run/taxonomy_second_run.tsv \
  --output-path EDENISS/Analysis_Paper_DADA2/taxonomy_second_run.qza \
  --input-format HeaderlessTSVTaxonomyFormat

<font size="2"> <p> Using the taxonomic classification from the previous step to remove reads stemming from mitochondria and chloroplasts from the feature table of the first sequencing run </p> </font>

In [ ]:
! $q; qiime taxa filter-table \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run.qza \
    --i-taxonomy  EDENISS/Analysis_Paper_DADA2/taxonomy_first_run.qza \
    --p-exclude mitochondria,chloroplast \
    --verbose \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run-filtered.qza

<font size="2"> <p> Filtering the feature table from the second run in the same way </p> </font>

In [ ]:
! $q; qiime taxa filter-table \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run.qza \
    --i-taxonomy  EDENISS/Analysis_Paper_DADA2/taxonomy_second_run.qza \
    --p-exclude mitochondria,chloroplast \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run-filtered.qza

Both taxonomies were then merged for later use

In [ ]:
! $q;  qiime feature-table merge-taxa \
    --i-data EDENISS/Analysis_Paper_DADA2/taxonomy_first_run.qza EDENISS/Analysis_Paper_DADA2/taxonomy_second_run.qza \
    --o-merged-data EDENISS/Analysis_Paper_DADA2/taxonomy-final.qza

# Quality control

The q2-quality-control plugin was used to remove contaminants introduced during DNA extraction and library preparation.

decontam-identify was used to calculate the likelihood that a read was the result of a contamination

In [ ]:
! $q; qiime quality-control decontam-identify \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run-filtered.qza \
    --m-metadata-file EDENISS/Metadata/decontam_metadata_DADA2.tsv \
    --p-method combined \
    --p-freq-concentration-column DNA-concentration \
    --p-prev-control-column sample-type \
    --p-prev-control-indicator control-blank \
    --o-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-first-run.qza 

decontam-score-viz was used to visualize the likelihood scores in a histogram to decide on a cutoff value for reads to be discarded

In [ ]:
! $q; qiime quality-control decontam-score-viz \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-first-run.qza   \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run-filtered.qza \
    --o-visualization EDENISS/Analysis_Paper_DADA2/decontam/hist-first-run.qzv

decontam-remove was used to filter out reads with values below the chosen threshold

In [ ]:
! $q; qiime quality-control decontam-remove \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-first-run.qza \
    --i-rep-seqs EDENISS/Analysis_Paper_DADA2/rep_seqs_first_run.qza \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run-filtered.qza \
    --p-threshold 0.1 \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run_decontam.qza \
    --o-filtered-rep-seqs EDENISS/Analysis_Paper_DADA2/rep_seqs_first_run_filterd_seqs_decontam.qza

The same process was used to filter the samples from the second run

In [ ]:
! $q; qiime quality-control decontam-identify \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run-filtered.qza \
    --m-metadata-file EDENISS/Metadata/decontam_metadata_FEG9_FEG10_DADA2.tsv \
    --p-method combined \
    --p-freq-concentration-column DNA-concentration \
    --p-prev-control-column sample-type \
    --p-prev-control-indicator control-blank \
    --o-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-second-run.qza 

In [ ]:
! $q; qiime quality-control decontam-score-viz \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-second-run.qza   \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run-filtered.qza \
    --o-visualization EDENISS/Analysis_Paper_DADA2/decontam/hist-second-run.qzv

In [ ]:
! $q; qiime quality-control decontam-remove \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-second-run.qza \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run-filtered.qza \
    --i-rep-seqs EDENISS/Analysis_Paper_DADA2/rep_seqs_second_run.qza \
    --p-threshold 0.4 \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run_decontam.qza \
    --o-filtered-rep-seqs EDENISS/Analysis_Paper_DADA2/rep_seqs_second_run_filterd_seqs_decontam.qza

The NTC and ZymoStandard columns were removed from both feature tables. Feature tables and repseqs from both runs were then merged 

In [ ]:
! $q; qiime tools import \
  --type 'FeatureTable[Frequency]' \
  --input-path EDENISS/Analysis_Paper_DADA2/import/feature_table_first_run_decontam_noctrl.biom \
  --output-path EDENISS/Analysis_Paper_DADA2/feature_table_first_run_decontam_noctrl.qza

In [ ]:
! $q; qiime tools import \
  --type 'FeatureTable[Frequency]' \
  --input-path EDENISS/Analysis_Paper_DADA2/import/feature_table_second_run_decontam_noctrl.biom \
  --output-path EDENISS/Analysis_Paper_DADA2/feature_table_second_run_decontam_noctrl.qza

In [ ]:
! $q; qiime feature-table merge \
    --i-tables EDENISS/Analysis_Paper_DADA2/feature_table_first_run_decontam_noctrl.qza EDENISS/Analysis_Paper_DADA2/feature_table_second_run_decontam_noctrl.qza \
    --o-merged-table EDENISS/Analysis_Paper_DADA2/feature-table-combined.qza

In [ ]:
! $q; qiime feature-table merge-seqs \
    --i-data EDENISS/Analysis_Paper_DADA2/rep_seqs_first_run_filterd_seqs_decontam.qza EDENISS/Analysis_Paper_DADA2/rep_seqs_second_run_filterd_seqs_decontam.qza \
    --o-merged-data EDENISS/Analysis_Paper_DADA2/rep-seqs-combined.qza

After merging the two sequencing runs the table was split into tables for pre-cleaning, post-cleaning and samples from the second campaign

In [ ]:
! $q; qiime feature-table split \
    --i-table EDENISS/Analysis_Paper_DADA2/feature-table-combined.qza \
    --m-metadata-file EDENISS/Metadata/Metadata_combined_DADA2.tsv \
    --m-metadata-column split_table \
    --o-tables EDENISS/Analysis_Paper_DADA2/split_tables

The pre- and post-cleaning samples were again filtered with decontam to remove contaminations that were caused during the sample aqcuisition. This step was skipped for the samples from the second campaign due to the lack of control samples.

Pre-cleaning

In [ ]:
! $q; qiime quality-control decontam-identify \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Pre.qza \
    --m-metadata-file EDENISS/Metadata/decontam_metadata_pre_DADA2.tsv \
    --p-method combined \
    --p-freq-concentration-column DNA-concentration \
    --p-prev-control-column sample-type \
    --p-prev-control-indicator control-sampling_pre \
    --o-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-pre-cleaning.qza 

In [ ]:
! $q; qiime quality-control decontam-score-viz \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-pre-cleaning.qza   \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Pre.qza \
    --o-visualization EDENISS/Analysis_Paper_DADA2/decontam/hist-pre-cleaning.qzv

In [ ]:
! $q; qiime quality-control decontam-remove \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-pre-filtered.qza \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Pre-filtered.qza \
    --i-rep-seqs EDENISS/Analysis_Paper_DADA2/rep-seqs-combined-filtered.qza \
    --p-threshold 0.14 \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature-table-pre-cleaning-filtered.qza \
    --o-filtered-rep-seqs EDENISS/Analysis_Paper_DADA2/rep-seqs-pre-cleaning-filtered.qza

Post-cleaning

In [ ]:
! $q; qiime quality-control decontam-identify \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Post.qza \
    --m-metadata-file EDENISS/Metadata/decontam_metadata_post_DADA2.tsv \
    --p-method combined \
    --p-freq-concentration-column DNA-concentration \
    --p-prev-control-column sample-type \
    --p-prev-control-indicator control-sampling_post \
    --o-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-post-cleaning.qza 

In [ ]:
! $q; qiime quality-control decontam-score-viz \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-post-cleaning.qza   \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Post.qza \
    --o-visualization EDENISS/Analysis_Paper_DADA2/decontam/hist-post-cleaning.qzv

In [ ]:
! $q; qiime quality-control decontam-remove \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-post-filtered.qza \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Post-filtered.qza \
    --i-rep-seqs EDENISS/Analysis_Paper_DADA2/rep-seqs-combined.qza \
    --p-threshold 0.18 \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature-table-post-cleaning-filtered.qza \
    --o-filtered-rep-seqs EDENISS/Analysis_Paper_DADA2/rep-seqs-post-cleaning-filtered.qza

After filtering the pre- and post-cleaning feature tables were merged again

In [ ]:
! $q; qiime feature-table merge \
    --i-tables EDENISS/Analysis_Paper_DADA2/feature-table-pre-cleaning-filtered.qza EDENISS/Analysis_Paper_DADA2/feature-table-post-cleaning-filtered.qza  \
    --o-merged-table EDENISS/Analysis_Paper_DADA2/feature-table-combined-filtered.qza

In [4]:
! $q; qiime feature-table merge \
    --i-tables EDENISS/Analysis_Paper_DADA2/feature-table-combined-filtered.qza EDENISS/Analysis_Paper_DADA2/split_tables/FEG9.qza  \
    --o-merged-table EDENISS/Analysis_Paper_DADA2/feature-table-all-filtered.qza

Saved FeatureTable[Frequency] to: EDENISS/Analysis_Paper_DADA2/feature-table-all-filtered.qza


In [ ]:
! $q; qiime taxa collapse \
  --i-table EDENISS/Analysis_Paper_DADA2/feature-table-combined-filtered.qza \
  --i-taxonomy EDENISS/Analysis_Paper_DADA2/taxonomy-final.qza \
  --p-level 6 \
  --o-collapsed-table EDENISS/Analysis_Paper_DADA2/feature-table-combined-filtered-genus.qza

In [ ]:
! $q; qiime taxa collapse \
  --i-table EDENISS/Analysis_Paper_DADA2/split_tables/FEG9.qza\
  --i-taxonomy EDENISS/Analysis_Paper_DADA2/taxonomy-final.qza \
  --p-level 6 \
  --o-collapsed-table EDENISS/Analysis_Paper_DADA2/feature-table-FEG9-genus.qza

# Rarefaction

Before alpha and beta diversity analysis could be carried out the feature tables needed to be rarefied

Due to the merging of the two sequencing runs it was necessary to construct a new phylogenetic tree for the pre- and post-cleaning samples

This was done by merging the filtered rep-seqs from the previous steps together

In [ ]:
! $q;  qiime feature-table merge-seqs\
    --i-data EDENISS/Analysis_Paper_DADA2/rep-seqs-post-cleaning-filtered.qza EDENISS/Analysis_Paper_DADA2/rep-seqs-pre-cleaning-filtered.qza \
    --o-merged-data EDENISS/Analysis_Paper_DADA2/rep_seqs_final.qza

The next step was the construction of a new phylogenetic tree with another preconstructed classifier

In [ ]:
! $q; qiime fragment-insertion sepp \
    --i-representative-sequences EDENISS/Analysis_Paper_DADA2/rep_seqs_final.qza \
    --i-reference-database EDENISS/Classifier/sepp-refs-gg-13-8.qza \
    --p-threads 4 \
    --o-tree EDENISS/Analysis_Paper_DADA2/insertion-tree.qza\
    --o-placements EDENISS/Analysis_Paper_DADA2/insertion-placements.qza

The phylogenetic tree was used to construct rarefaction curves for the pre- and post-cleaning and the FEG9 samples

In [ ]:
!$q; qiime diversity alpha-rarefaction \
    --i-table EDENISS/Analysis_Paper_DADA2/feature-table-combined-filtered.qza \
    --i-phylogeny EDENISS/Analysis_Paper_DADA2/insertion-tree.qza \
    --m-metadata-file EDENISS/Metadata/Metadata_combined_DADA2.tsv \
    --p-max-depth 8267 \
    --p-min-depth 100 \
    --p-steps 20 \
    --p-iterations 10 \
    --o-visualization EDENISS/Analysis_Paper_DADA2/alpha_diversity/rarefaction_curves_prepost.qzv

In [ ]:
!$q; qiime diversity alpha-rarefaction \
    --i-table EDENISS/Analysis_Paper/feature-table-FEG9.qza \
    --i-phylogeny EDENISS/Analysis_Paper/insertion-tree.qza \
    --m-metadata-file EDENISS/Metadata/Metadata_combined_DADA2.tsv \
    --p-max-depth 2156 \
    --p-min-depth 100 \
    --p-steps 20 \
    --p-iterations 10 \
    --o-visualization EDENISS/Analysis_Paper/alpha_diversity/rarefaction_curves_FEG9.qzv

Looking at the rarefaction curves allows for a visual estimation of a viable rarefaction depth. In this case 1200 was chosen for the pre- and post-cleaning samples and 900 for the FEG9 samples

Pre- and post-cleaning samples

In [ ]:
! $q; qiime feature-table rarefy \
    --i-table EDENISS/Analysis_Paper_DADA2/feature-table-combined-filtered.qza \
    --p-sampling-depth 1200 \
    --o-rarefied-table EDENISS/Analysis_Paper_DADA2/alpha_diversity/rarefied_table_prepost_1200.qza

FEG9 samples

In [ ]:
! $q; qiime feature-table rarefy \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/FEG9.qza \
    --p-sampling-depth 900 \
    --o-rarefied-table EDENISS/Analysis_Paper_DADA2/alpha_diversity/rarefied_table_FEG9_900.qza

# Alpha Diversity

Three different alpha diversity measures were calculated: Observed features, Shannon index and Faith's PD

Pre- and post-cleaning samples

In [ ]:
! $q; qiime diversity alpha \
    --i-table EDENISS/Analysis_Paper_DADA2/alpha_diversity/rarefied_table_prepost_1200.qza \
    --p-metric 'goods_coverage' \
    --o-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/goods_prepost_1200.qza

In [ ]:
! $q; qiime diversity alpha-group-significance \
    --i-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/goods_prepost_1200.qza \
    --m-metadata-file EDENISS/Metadata/Metadata_combined_DADA2.tsv \
    --o-visualization EDENISS/Analysis_Paper_DADA2/alpha_diversity/goods_prepost_visualization.qzv

In [ ]:
! $q; qiime diversity alpha \
    --i-table EDENISS/Analysis_Paper_DADA2/alpha_diversity/rarefied_table_prepost_1200.qza \
    --p-metric 'chao1' \
    --o-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/chao1_prepost_1200.qza

In [ ]:
! $q; qiime diversity alpha-group-significance \
    --i-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/chao1_prepost_1200.qza \
    --m-metadata-file EDENISS/Metadata/Metadata_combined_DADA2.tsv \
    --o-visualization EDENISS/Analysis_Paper_DADA2/alpha_diversity/chao1_prepost_visualization.qzv

In [ ]:
! $q; qiime diversity alpha \
    --i-table EDENISS/Analysis_Paper_DADA2/alpha_diversity/rarefied_table_prepost_1200.qza \
    --p-metric 'shannon' \
    --o-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/shannon_prepost_1200.qza

In [ ]:
! $q; qiime diversity alpha-group-significance \
    --i-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/shannon_prepost_1200.qza \
    --m-metadata-file EDENISS/Metadata/Metadata_combined_DADA2.tsv \
    --o-visualization EDENISS/Analysis_Paper_DADA2/alpha_diversity/shannon_prepost_visualization.qzv

FEG9 samples

In [ ]:
! $q; qiime diversity alpha \
    --i-table EDENISS/Analysis_Paper_DADA2/alpha_diversity/rarefied_table_FEG9_900.qza \
    --p-metric 'goods_coverage' \
    --o-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/goods_FEG9_900.qza

In [ ]:
! $q; qiime diversity alpha-group-significance \
    --i-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/goods_FEG9_900.qza \
    --m-metadata-file EDENISS/Metadata/Metadata_FEG9.tsv \
    --o-visualization EDENISS/Analysis_Paper_DADA2/alpha_diversity/goods_FEG9_visualization.qzv

In [ ]:
! $q; qiime diversity alpha \
    --i-table EDENISS/Analysis_Paper_DADA2/alpha_diversity/rarefied_table_FEG9_900.qza \
    --p-metric 'chao1' \
    --o-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/chao1_FEG9_900.qza

In [ ]:
! $q; qiime diversity alpha-group-significance \
    --i-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/chao1_FEG9_900.qza \
    --m-metadata-file EDENISS/Metadata/Metadata_FEG9.tsv \
    --o-visualization EDENISS/Analysis_Paper_DADA2/alpha_diversity/chao1_FEG9_visualization.qzv

In [ ]:
! $q; qiime diversity alpha \
    --i-table EDENISS/Analysis_Paper_DADA2/alpha_diversity/rarefied_table_FEG9_900.qza \
    --p-metric 'shannon' \
    --o-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/shannon_FEG9_900.qza

In [ ]:
! $q; qiime diversity alpha-group-significance \
    --i-alpha-diversity EDENISS/Analysis_Paper_DADA2/alpha_diversity/shannon_FEG9_900.qza \
    --m-metadata-file EDENISS/Metadata/Metadata_FEG9.tsv \
    --o-visualization EDENISS/Analysis_Paper_DADA2/alpha_diversity/shannon_FEG9_visualization.qzv

# Beta Diversity

Beta diversity was calculated using the Gemelli plugin to create Beta diversity biplots

Pre- and post-cleaning samples

In [ ]:
! $q; qiime gemelli rpca \
    --i-table EDENISS/Analysis_Paper_DADA2/alpha_diversity/rarefied_table_prepost_1200.qza \
    --p-min-sample-count 10 \
    --o-biplot EDENISS/Analysis_Paper_DADA2/gemelli/gemelli_ordination_prepost_1200.qza \
    --o-distance-matrix EDENISS/Analysis_Paper_DADA2/gemelli/gemelli_distance_prepost_1200.qza

In [ ]:
! $q; qiime emperor biplot\
    --i-biplot EDENISS/Analysis_Paper_DADA2/gemelli/gemelli_ordination_prepost_1200.qza \
    --m-sample-metadata-file EDENISS/Metadata/Metadata_combined_DADA2.tsv \
    --m-feature-metadata-file EDENISS/Analysis_Paper_DADA2/taxonomy-final.qza \
    --o-visualization EDENISS/Analysis_Paper_DADA2/gemelli/RPCA-biplot-prepost_1200.qzv  

FEG9 samples

In [ ]:
! $q; qiime gemelli rpca \
    --i-table EDENISS/Analysis_Paper_DADA2/alpha_diversity/rarefied_table_FEG9_900.qza \
    --p-min-sample-count 10 \
    --o-biplot EDENISS/Analysis_Paper_DADA2/gemelli/gemelli_ordination_FEG9_900.qza \
    --o-distance-matrix EDENISS/Analysis_Paper/gemelli/gemelli_distance_FEG9_900.qza

In [53]:
! $q; qiime emperor biplot\
    --i-biplot EDENISS/Analysis_Paper_DADA2/gemelli/gemelli_ordination_FEG9_900.qza \
    --m-sample-metadata-file EDENISS/Metadata/Metadata_combined_DADA2.tsv \
    --m-feature-metadata-file EDENISS/Analysis_Paper_DADA2/taxonomy-final.qza \
    --o-visualization EDENISS/Analysis_Paper_DADA2/gemelli/RPCA-biplot-FEG9_900.qzv 

Traceback (most recent call last):
  File "/homes/lgaiser/miniconda3/envs/q2dev-amplicon/lib/python3.9/site-packages/q2cli/util.py", line 333, in _load_metadata_artifact
    return artifact.view(qiime2.Metadata), None
  File "/homes/lgaiser/miniconda3/envs/q2dev-amplicon/lib/python3.9/site-packages/qiime2/sdk/result.py", line 374, in view
    return self._view(view_type)
  File "/homes/lgaiser/miniconda3/envs/q2dev-amplicon/lib/python3.9/site-packages/qiime2/sdk/result.py", line 406, in _view
    result = transformation(self._archiver.data_dir)
  File "/homes/lgaiser/miniconda3/envs/q2dev-amplicon/lib/python3.9/site-packages/qiime2/core/transform.py", line 70, in transformation
    new_view = transformer(view)
  File "/homes/lgaiser/miniconda3/envs/q2dev-amplicon/lib/python3.9/site-packages/qiime2/core/transform.py", line 214, in wrapped
    return transformer(view.file.view(self._wrapped_view_type))
  File "/homes/lgaiser/miniconda3/envs/q2dev-amplicon/lib/python3.9/site-packages/q2_t